In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('concatenated_data.csv')

In [2]:
location_data = {
    1: [39.98966406, 116.403458],
    2: [40.22994018, 116.2330318],
    3: [40.30023118, 116.2324668],
    4: [39.93865024, 116.4350122],
    5: [39.93736379, 116.3693659],
    6: [39.91345006, 116.1956398],
    7: [40.30842471, 116.6469836],
    8: [39.94700908, 116.470307],
    9: [40.17700804, 116.664263],
    10: [39.88797708, 116.417313],
    11: [39.9727465, 116.3056615],
    12: [39.88565298, 116.3747278]
}

location_df = pd.DataFrame.from_dict(location_data, orient='index', columns=['Latitude', 'Longitude'])

df_merged = df.merge(location_df, left_on='station', right_index=True)

In [3]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import branca
import branca.colormap as cm

# Group the DataFrame by 'station' and get the last row of each group
last_rows = df_merged.groupby('station').tail(1)

map_center = [39.995, 116.40]
map_zoom = 9

map_beijing = folium.Map(location=map_center, zoom_start=map_zoom)

# Filter the DataFrame for rows, then columns, then remove NaNs
heat_df = last_rows[['Latitude', 'Longitude', 'PM2.5']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude', 'Longitude', 'PM2.5'])

# List comprehension to make our list of lists
heat_data = [[row['Latitude'], row['Longitude'], row['PM2.5']] for index, row in heat_df.iterrows()]


# Plot the heatmap with the custom gradient
HeatMap(heat_data, radius=30).add_to(map_beijing)


# Plot it on the map
for point in heat_data:
    lat, lon, value = point
    folium.Marker(
    location=[lat, lon],
    icon=None,
    popup=f"Value: {value}").add_to(map_beijing)


'''
# Add the color scale legend to the map
color_scale.caption = 'PM2.5'
map_beijing.add_child(color_scale)
'''

# Display the map
map_beijing

In [4]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import branca
import branca.colormap as cm

# Group the DataFrame by 'station' and get the last row of each group
last_rows = df_merged.groupby('station').tail(1)

map_center = [39.995, 116.40]
map_zoom = 9

map_beijing = folium.Map(location=map_center, zoom_start=map_zoom)

# Filter the DataFrame for rows, then columns, then remove NaNs
last_rows = last_rows.loc[last_rows['station'].isin([2, 4, 6, 7])]
heat_df = last_rows[['Latitude', 'Longitude', 'PM2.5']]
heat_df = heat_df.dropna(axis=0, subset=['Latitude', 'Longitude', 'PM2.5'])

# List comprehension to make our list of lists
heat_data = [[row['Latitude'], row['Longitude'], row['PM2.5']] for index, row in heat_df.iterrows()]


# Plot the heatmap with the custom gradient
HeatMap(heat_data, radius=30).add_to(map_beijing)


# Plot it on the map
for point in heat_data:
    lat, lon, value = point
    folium.Marker(
    location=[lat, lon],
    icon=None,
    popup=f"Value: {value}").add_to(map_beijing)



'''
# Add the color scale legend to the map
color_scale.caption = 'PM2.5'
map_beijing.add_child(color_scale)
'''

# Display the map
map_beijing

In [7]:
import pandas as pd
import folium
from folium import plugins
import datetime
import warnings

warnings.filterwarnings('ignore')
df_merged['datetime'] = pd.to_datetime(df_merged[['year', 'month', 'day', 'hour']])
# Replace with your DataFrame containing the required attributes
df = df_merged[(df_merged['station'].isin([2, 4, 6, 7])) & (df_merged['datetime'] >= '2016-12-01') & (df_merged['datetime'] <= '2017-02-01')]

# Assuming the 'year', 'month', 'day', and 'hour' columns are integers
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

# Select the data for the last 24 hours
end_time = df['datetime'].max()
start_time = end_time - pd.Timedelta(hours=24)
df_last_24h = df[(df['datetime'] >= start_time) & (df['datetime'] <= end_time)]

map_center = [39.995, 116.40]
map_zoom = 9

map_beijing = folium.Map(location=map_center, zoom_start=map_zoom)

# Create heat_data as a list of lists for each hour
heat_data = []
for hour in pd.date_range('2016-12-01', '2017-02-01', freq='H'):
    hourly_df = df[df['datetime'] == hour]
    # Use logarithmic scaling to compress the range of PM2.5 values
    #hourly_df['log_PM2.5'] = np.log10(hourly_df['PM2.5'])
    min_pm25 = hourly_df['PM2.5'].min()
    max_pm25 = hourly_df['PM2.5'].max()
    hourly_df.loc[:, 'norm_PM2.5'] = (hourly_df['PM2.5'] - min_pm25) / (max_pm25 - min_pm25)
    heat_data.append([[row['Latitude'], row['Longitude'], row['norm_PM2.5']] for index, row in hourly_df.iterrows()])

        

# Create the HeatMapWithTime and add it to the map
hm = plugins.HeatMapWithTime(heat_data, radius=30, auto_play=True, max_opacity=0.8)
hm.add_to(map_beijing)

# Display the map
map_beijing


In [8]:
import pandas as pd
import folium
from folium import plugins
import datetime
import warnings

warnings.filterwarnings('ignore')

# Replace with your DataFrame containing the required attributes
df = df_merged

# Assuming the 'year', 'month', 'day', and 'hour' columns are integers
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

# Select the data for the last 24 hours
end_time = df['datetime'].max()
start_time = end_time - pd.Timedelta(hours=24)
df_last_24h = df[(df['datetime'] >= start_time) & (df['datetime'] <= end_time)]

map_center = [39.995, 116.40]
map_zoom = 9

map_beijing = folium.Map(location=map_center, zoom_start=map_zoom)

# Create heat_data as a list of lists for each hour
heat_data = []
for hour in pd.date_range(start_time, end_time, freq='H'):
    hourly_df = df_last_24h[df_last_24h['datetime'] == hour]
    # Use logarithmic scaling to compress the range of PM2.5 values
    #hourly_df['log_PM2.5'] = np.log10(hourly_df['PM2.5'])
    min_pm25 = hourly_df['PM2.5'].min()
    max_pm25 = hourly_df['PM2.5'].max()
    hourly_df.loc[:, 'norm_PM2.5'] = (hourly_df['PM2.5'] - min_pm25) / (max_pm25 - min_pm25)
    heat_data.append([[row['Latitude'], row['Longitude'], row['norm_PM2.5']] for index, row in hourly_df.iterrows()])

        

# Create the HeatMapWithTime and add it to the map
hm = plugins.HeatMapWithTime(heat_data, radius=30, auto_play=True, max_opacity=0.8)
hm.add_to(map_beijing)

# Display the map
map_beijing
